# Curacao Risk Level for Extreme Sea Level Hazard Using Gornitz et al. (1994)
# Curacao Multi-Criteria Risk Level for Extreme Sea Level Hazard Using Mclaughlin and Cooper (2010)


In [181]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from scipy.spatial import KDTree
import numpy as np

# Additional imports
import pystac_client
import xarray as xr
import pathlib
import os
import geopandas as gpd
import matplotlib as mpl
from matplotlib import colors
import rasterio

import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
from rasterio.plot import show

import matplotlib.colors as mcolors
from shapely.geometry import Point
import pandas as pd

from tqdm import tqdm

import webbrowser
from pathlib import Path
import os
import time

from itertools import compress
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm

from shapely.geometry import box



In [182]:
Exposure_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Exposure\Caribbean_Exposure_classified.csv")
Exposure_df["Sediment_Type - Exposure level"]=Exposure_df["Sediment Type"]


Exposure_df

,transect_id,country_id,continent,country_name,Percentage of Built Up Area,LECZ Area÷LECZ mean elevation,Neashore Slope (tanβ or m^-1),Population Count Within 1km,Sediment Type,geometry,Percentage of Built Up Area - Exposure level,LECZ Area÷LECZ mean elevation - Exposure level,Neashore Slope (tanβ or m^-1) - Exposure level,Population Count Within 1km - Exposure level,Sediment_Type - Exposure level
0,BOX_116_353_2,COL,South America,Colombia,0.0,1.366501e+06,11.291917,0.0,4.0,POINT (-71.8798045985 12.250683718),1,2,4.0,1,4.0
1,BOX_116_353_3,COL,South America,Colombia,0.0,1.974146e+06,13.060951,0.0,2.0,POINT (-71.87698143850001 12.247308305),1,3,4.0,1,2.0
2,BOX_116_353_4,COL,South America,Colombia,0.0,3.834474e+06,13.060951,0.0,4.0,POINT (-71.8756318462 12.2431443263),1,4,4.0,1,4.0
3,BOX_116_353_5,COL,South America,Colombia,0.0,5.239073e+06,11.420912,0.0,2.0,POINT (-71.8752334021 12.2387721729),1,4,4.0,1,2.0
4,BOX_116_353_6,COL,South America,Colombia,0.0,6.112962e+06,11.420912,0.0,2.0,POINT (-71.8748349513 12.2343999454),1,5,4.0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,0.0,1.793014e+06,3.867398,1593.0,4.0,POINT (-71.6681862112 17.9652589019),1,2,3.0,2,4.0
37505,BOX_139_002_62,DOM,North America,Dominican Republic,0.0,1.861771e+06,3.867398,2227.0,4.0,POINT (-71.6662284163 17.9614175026),1,2,3.0,2,4.0
37506,BOX_139_002_63,DOM,North America,Dominican Republic,0.0,1.970466e+06,3.790441,2710.0,4.0,POINT (-71.664582619 17.9574419454),1,3,3.0,2,4.0
37507,BOX_139_002_64,DOM,North America,Dominican Republic,0.0,2.126400e+06,3.790441,2617.0,4.0,POINT (-71.6629367897 17.953466310899998),1,3,3.0,2,4.0


In [183]:
Hazard_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Hazard\Present\Caribbean_Hazards_classified.csv")
Hazard_df

,transect_id,country_id,continent,country_name,ESL_100yrRP,EWH_100yrRP,Land_Subsidence_2010,Shoreline_Change_Rate_PC,geometry,ESL_100yrRP - Hazard level,EWH_100yrRP - Hazard level,Land_Subsidence_2010 - Hazard level,Shoreline_Change_Rate_PC - Hazard level
0,BOX_116_353_2,COL,South America,Colombia,0.946771,4.455543,NaN,-16.441771,POINT (-71.8798045985 12.250683718),1.0,1,NaN,5.0
1,BOX_116_353_3,COL,South America,Colombia,0.946771,4.455543,NaN,0.335379,POINT (-71.87698143850001 12.247308305),1.0,1,NaN,2.0
2,BOX_116_353_4,COL,South America,Colombia,0.946771,4.455543,NaN,1.878836,POINT (-71.8756318462 12.2431443263),1.0,1,NaN,1.0
3,BOX_116_353_5,COL,South America,Colombia,0.946771,4.455543,NaN,0.490707,POINT (-71.8752334021 12.2387721729),1.0,1,NaN,2.0
4,BOX_116_353_6,COL,South America,Colombia,0.946771,4.455543,NaN,-0.052123,POINT (-71.8748349513 12.2343999454),1.0,1,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,1.246410,22.563481,2.0,0.288753,POINT (-71.6681862112 17.9652589019),2.0,4,2.0,2.0
37505,BOX_139_002_62,DOM,North America,Dominican Republic,1.246410,22.563481,2.0,0.166001,POINT (-71.6662284163 17.9614175026),2.0,4,2.0,2.0
37506,BOX_139_002_63,DOM,North America,Dominican Republic,1.246410,22.563481,2.0,0.092415,POINT (-71.664582619 17.9574419454),2.0,4,2.0,2.0
37507,BOX_139_002_64,DOM,North America,Dominican Republic,1.246410,22.563481,2.0,-0.027472,POINT (-71.6629367897 17.953466310899998),2.0,4,2.0,2.0


Curacao Risk Level for Extreme Sea Level Hazard Using Gornitz et al. (1994) CVI6 

In [184]:
# UNCOMMENT AS NEEDED

# gornitz_hazard_used="Extreme Sea Level - PC - Hazard level"
# gornitz_hazard_used="Sig. WaveHeight 95th% - PC - Hazard level"
gornitz_hazard_used="Land_Subsidence_2010 - Hazard level"
units=""


Value_to_show=title_name=gornitz_hazard_used.replace(" - Hazard level","")

save_directory=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Risk\01 Present"
Haz_Component= Hazard_df[Value_to_show].apply(lambda x: f"{x:.2f} {units}")



title_name=gornitz_hazard_used.replace(" - Hazard level","")

gornitz_ESL=pd.concat([Hazard_df.iloc[:,0],Hazard_df[gornitz_hazard_used],Exposure_df.iloc[:,-5:]],axis=1)
gornitz_ESL

,transect_id,Land_Subsidence_2010 - Hazard level,Percentage of Built Up Area - Exposure level,LECZ Area÷LECZ mean elevation - Exposure level,Neashore Slope (tanβ or m^-1) - Exposure level,Population Count Within 1km - Exposure level,Sediment_Type - Exposure level
0,BOX_116_353_2,NaN,1,2,4.0,1,4.0
1,BOX_116_353_3,NaN,1,3,4.0,1,2.0
2,BOX_116_353_4,NaN,1,4,4.0,1,4.0
3,BOX_116_353_5,NaN,1,4,4.0,1,2.0
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0
...,...,...,...,...,...,...,...
37504,BOX_139_002_61,2.0,1,2,3.0,2,4.0
37505,BOX_139_002_62,2.0,1,2,3.0,2,4.0
37506,BOX_139_002_63,2.0,1,3,3.0,2,4.0
37507,BOX_139_002_64,2.0,1,3,3.0,2,4.0


In [185]:
indicator_num=len(gornitz_ESL.columns)-1
indicator_num

6

In [186]:
gornitz_ESL_filled=gornitz_ESL.copy()
gornitz_ESL_filled["Risk Level"]=(gornitz_ESL[gornitz_hazard_used]+(np.nansum(gornitz_ESL.iloc[:,-5:],axis=1)/len(gornitz_ESL.iloc[0,-5:])))/2
gornitz_ESL_filled["Risk Level class"] = np.ceil(gornitz_ESL_filled["Risk Level"])
gornitz_ESL_filled

,transect_id,Land_Subsidence_2010 - Hazard level,Percentage of Built Up Area - Exposure level,LECZ Area÷LECZ mean elevation - Exposure level,Neashore Slope (tanβ or m^-1) - Exposure level,Population Count Within 1km - Exposure level,Sediment_Type - Exposure level,Risk Level,Risk Level class
0,BOX_116_353_2,NaN,1,2,4.0,1,4.0,NaN,NaN
1,BOX_116_353_3,NaN,1,3,4.0,1,2.0,NaN,NaN
2,BOX_116_353_4,NaN,1,4,4.0,1,4.0,NaN,NaN
3,BOX_116_353_5,NaN,1,4,4.0,1,2.0,NaN,NaN
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,2.0,1,2,3.0,2,4.0,2.2,3.0
37505,BOX_139_002_62,2.0,1,2,3.0,2,4.0,2.2,3.0
37506,BOX_139_002_63,2.0,1,3,3.0,2,4.0,2.3,3.0
37507,BOX_139_002_64,2.0,1,3,3.0,2,4.0,2.3,3.0


In [187]:


df = gornitz_ESL.iloc[:,1:]

# Identify columns with NaN values
nan_columns = df.columns[df.isna().any()].tolist()

# Create a new column 'NaN_Columns' containing list of column names with NaN values for each row
gornitz_ESL_filled['Uncertainty_Columns'] = df.apply(lambda row: [col for col in nan_columns if pd.isna(row[col])], axis=1)
gornitz_ESL_filled['Uncertainty_Columns_length'] = gornitz_ESL_filled['Uncertainty_Columns'].apply(len)

df


,Land_Subsidence_2010 - Hazard level,Percentage of Built Up Area - Exposure level,LECZ Area÷LECZ mean elevation - Exposure level,Neashore Slope (tanβ or m^-1) - Exposure level,Population Count Within 1km - Exposure level,Sediment_Type - Exposure level
0,NaN,1,2,4.0,1,4.0
1,NaN,1,3,4.0,1,2.0
2,NaN,1,4,4.0,1,4.0
3,NaN,1,4,4.0,1,2.0
4,NaN,1,5,4.0,1,2.0
...,...,...,...,...,...,...
37504,2.0,1,2,3.0,2,4.0
37505,2.0,1,2,3.0,2,4.0
37506,2.0,1,3,3.0,2,4.0
37507,2.0,1,3,3.0,2,4.0


In [188]:

gornitz_ESL_filled["Confindence %"]=100*(indicator_num-gornitz_ESL_filled['Uncertainty_Columns'].apply(len))/indicator_num
gornitz_ESL_filled

,transect_id,Land_Subsidence_2010 - Hazard level,Percentage of Built Up Area - Exposure level,LECZ Area÷LECZ mean elevation - Exposure level,Neashore Slope (tanβ or m^-1) - Exposure level,Population Count Within 1km - Exposure level,Sediment_Type - Exposure level,Risk Level,Risk Level class,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %
0,BOX_116_353_2,NaN,1,2,4.0,1,4.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333
1,BOX_116_353_3,NaN,1,3,4.0,1,2.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333
2,BOX_116_353_4,NaN,1,4,4.0,1,4.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333
3,BOX_116_353_5,NaN,1,4,4.0,1,2.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,2.0,1,2,3.0,2,4.0,2.2,3.0,[],0,100.000000
37505,BOX_139_002_62,2.0,1,2,3.0,2,4.0,2.2,3.0,[],0,100.000000
37506,BOX_139_002_63,2.0,1,3,3.0,2,4.0,2.3,3.0,[],0,100.000000
37507,BOX_139_002_64,2.0,1,3,3.0,2,4.0,2.3,3.0,[],0,100.000000


In [189]:
# Read the CSV file into a DataFrame
circle_center_df = Exposure_df


# Identify the correct geometry column (replace 'geometry_column_name' with the actual column name)
geometry_column_name = 'geometry'
circle_center_df['x'] = circle_center_df[geometry_column_name].apply(lambda point_str: float(point_str.strip('POINT ()').split()[0]))
circle_center_df['y'] = circle_center_df[geometry_column_name].apply(lambda point_str: float(point_str.strip('POINT ()').split()[1]))

circle_center_df = circle_center_df.drop(columns=[geometry_column_name])

# Create the GeoDataFrame with the correct geometry column
gornitz_ESL_filled_gdf = gpd.GeoDataFrame(gornitz_ESL_filled, geometry=gpd.points_from_xy(circle_center_df.x, circle_center_df.y))



gornitz_ESL_filled_gdf

,transect_id,Land_Subsidence_2010 - Hazard level,Percentage of Built Up Area - Exposure level,LECZ Area÷LECZ mean elevation - Exposure level,Neashore Slope (tanβ or m^-1) - Exposure level,Population Count Within 1km - Exposure level,Sediment_Type - Exposure level,Risk Level,Risk Level class,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %,geometry
0,BOX_116_353_2,NaN,1,2,4.0,1,4.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333,POINT (-71.87980 12.25068)
1,BOX_116_353_3,NaN,1,3,4.0,1,2.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333,POINT (-71.87698 12.24731)
2,BOX_116_353_4,NaN,1,4,4.0,1,4.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333,POINT (-71.87563 12.24314)
3,BOX_116_353_5,NaN,1,4,4.0,1,2.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333,POINT (-71.87523 12.23877)
4,BOX_116_353_6,NaN,1,5,4.0,1,2.0,NaN,NaN,[Land_Subsidence_2010 - Hazard level],1,83.333333,POINT (-71.87483 12.23440)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,2.0,1,2,3.0,2,4.0,2.2,3.0,[],0,100.000000,POINT (-71.66819 17.96526)
37505,BOX_139_002_62,2.0,1,2,3.0,2,4.0,2.2,3.0,[],0,100.000000,POINT (-71.66623 17.96142)
37506,BOX_139_002_63,2.0,1,3,3.0,2,4.0,2.3,3.0,[],0,100.000000,POINT (-71.66458 17.95744)
37507,BOX_139_002_64,2.0,1,3,3.0,2,4.0,2.3,3.0,[],0,100.000000,POINT (-71.66294 17.95347)


In [190]:
# import os
# from dotenv import load_dotenv

# # Load the .env file
# load_dotenv(r'C:\Users\rowe\Documents\GitHub\coclicodata\.env')

# # Access the Mapbox Access Token
# mapbox_access_token = os.getenv('MAPBOX_ACCESS_TOKEN')

# print("Mapbox Access Token:", mapbox_access_token)

In [191]:
img_lat = 18.1
img_lon = -77.3
view_width = 10

In [192]:
directory_path=r"C:\Users\rowe\Documents\GitHub\global-coastal-atlas\STAC\visualization\SOTC_transect_mapping_visual\Caribbean\Caribbean Plots\Risk_Levels"
long_name="Risk Level class"
# color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']
label_dict = {
1: "Very Low",
2: "Low",
3: "Medium",
4: "High",
5: "Very High"}

colors = {'Very Low': '#33f510', 'Low': '#3CB371', 'Medium': '#FFFF00', 'High': '#FFA500', 'Very High': '#FF4500'}

gornitz_ESL_filled_gdf[Value_to_show] = Haz_Component

#Sorted for Risk level view
sorted_CCN=gornitz_ESL_filled_gdf.sort_values(long_name)


# Create scatter map plot
fig_region = px.scatter_mapbox(sorted_CCN,
                        lat=sorted_CCN["geometry"].y,
                        lon=sorted_CCN["geometry"].x,
                        # hover_name=sorted_CCN[long_name].replace(label_dict),
                        hover_name=Value_to_show,
                        color=sorted_CCN[long_name].replace(label_dict),
                        color_discrete_map=colors,
                        mapbox_style="carto-positron",width=2000,
                        height=1200)


# Update map layout to adjust center and zoom
fig_region.update_layout(mapbox=dict(center=dict(lat=img_lat + 1, lon=img_lon + 4), zoom=5.5),
                title=f"{gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk')} Distribution along Caribbean's shoreline")

fig_region .update_layout(legend=dict(
traceorder='normal',
title=long_name,
title_font=dict(size=16),
itemsizing='constant',))
fig_region.show()


fig_name_shoreline=gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk') +"- Caribbean.png"
pio.write_image(fig_region, os.path.join(directory_path, fig_name_shoreline))

In [193]:
gornitz_ESL_filled_gdf.to_csv(save_directory+f"/{gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk')}.csv",index=False)

In [194]:
# color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']


# fig2 = go.Figure(go.Scattermapbox(
#     lat=gornitz_ESL_filled_gdf["geometry"].y,
#     lon=gornitz_ESL_filled_gdf["geometry"].x,
#     mode='markers',
#     marker=dict(
#         size=10,
#         color=gornitz_ESL_filled_gdf["Risk Level"],  # Set color to red

#         colorscale=color_scale,  # Specify the discrete color scale
#         cmin=1,  # Set the minimum value for the color scale
#         cmax=5,  # Set the maximum value for the color scale
#         colorbar=dict(
#             # title=""  # Set color bar title?
#             title = "Risk Level", 
#             tickvals=[1, 2, 3, 4, 5],  # Set the tick values
#             ticktext=['1', '2', '3', '4', '5'],  # Set the tick labels    
#         )
#     ),
#     hoverinfo='text',
#     hovertext=gornitz_ESL_filled_gdf["Uncertainty_Columns"],
#     showlegend=False
# ))

# # Update layout properties
# fig2.update_layout(
#     title = "Curacao Risk Level for " +title_name+" Using Gornitz et al. (1994) CVI6 ",
#     mapbox_style="carto-positron",
#     mapbox=dict(
#         zoom=9.5,
#         center=dict(lat=12.18625, lon=-68.95)
#     ),		

#     margin=dict(l=0, r=50, b=0, t=100),
#     legend=dict(
#     x=1.15,
#     y=1.0,
#     traceorder="normal",
#     orientation="v"
# ),
#     width=800,  # Set width of the plot
#     height=600  # Set height of the plot
# )

# # Show the plot
# fig2.show()

# directory_path = r"C:\Users\rowe\Documents\GitHub\global-coastal-atlas\STAC\visualization\SOTC_transect_mapping_visual\Plots\Poster\FINAL-CVI"
# pio.write_image(fig2, os.path.join(directory_path,gornitz_ESL_filled_gdf.columns[1].replace('Hazard', 'Risk') +"- Curacao.png"))

In [195]:
# import plotly.graph_objects as go

# color_scale = ['#33f510', '#3CB371', '#FFFF00', '#FFA500', '#FF4500']

# fig2 = go.Figure(go.Scattermapbox(
#     lat=gornitz_ESL_filled_gdf["geometry"].y,
#     lon=gornitz_ESL_filled_gdf["geometry"].x,
#     mode='markers',
#     marker=dict(
#         size=15,
#         color=gornitz_ESL_filled_gdf["Risk Level"],  # Set color to red
#         colorscale=color_scale,  # Specify the discrete color scale
#         cmin=1,  # Set the minimum value for the color scale
#         cmax=5,  # Set the maximum value for the color scale
#         colorbar=dict(
#             title="Risk Level",
#             tickvals=[1, 2, 3, 4, 5],  # Set the tick values
#             ticktext=['1', '2', '3', '4', '5'],  # Set the tick labels
#         )
#     ),
#     hoverinfo='text',
#     hovertext=gornitz_ESL_filled_gdf["Uncertainty_Columns"],
#     showlegend=False
# ))

# # Update layout properties
# fig2.update_layout(
#     title="Curacao Risk Level for Extreme Sea Level Hazard Using Gornitz et al. (1994) CVI6",
#     mapbox=dict(
#         accesstoken=mapbox_access_token,  # Set the Mapbox access token here
#         style="satellite-streets",
#         zoom=5,
#         center=dict(lat=12.18625, lon=-68.95)
#     ),
#     margin=dict(l=0, r=50, b=0, t=100),
#     legend=dict(
#         x=1.15,
#         y=1.0,
#         traceorder="normal",
#         orientation="v"
#     ),
#     width=800,  # Set width of the plot
#     height=600  # Set height of the plot
# )

# # Show the plot
# fig2.show()


Curacao Multi-Criteria Risk Level for Extreme Sea Level Hazard Using Mclaughlin and Cooper (2010)